Install library for QC

In [481]:
%pip install dwave-ocean-sdk
%pip install dwave-neal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Подключим все необходимые библиотеки

In [482]:
import random
import math
import itertools

import dimod
from dimod import BinaryQuadraticModel
from dimod.reference.samplers import ExactSolver
from neal import SimulatedAnnealingSampler

Сгенерируем случайные данные

In [483]:
# ClientsNum = random.randint(1, 3)
ClientsNum = 6
DepotsNum = random.randint(1, 1)
CarsNum = random.randint(DepotsNum, DepotsNum+1)

Distances = [[0 for _ in range(ClientsNum + DepotsNum)] for _ in range(ClientsNum + DepotsNum)]
for i in range(ClientsNum + DepotsNum):
    for j in range(i + 1, ClientsNum + DepotsNum):
        Distances[i][j] = random.randint(100, 5000)
        Distances[j][i] = Distances[i][j]

# GoodsDepot = [random.randint(50, 100) for _ in range(DepotsNum)]
# GoodsCar = [random.randint(50, 100) for _ in range(CarsNum)]
# GoodsClients = [random.randint(1, 9) for _ in range(ClientsNum)]

CarDepot = [[0 for _ in range(DepotsNum)] for _ in range(CarsNum)]
CarDepotMap = {}
for i in range(CarsNum):
    depot = random.randint(0, DepotsNum - 1)
    CarDepot[i][depot] = 1
    CarDepotMap[i] = depot

# Optimized parameters amount

xNum = ClientsNum * ClientsNum * CarsNum
uNum = ClientsNum * CarsNum
nNum = ClientsNum * CarsNum

print("Total Amount of Parameters: ", xNum + uNum + nNum)

Total Amount of Parameters:  96


Инициализируем матрицу QUBO

In [484]:
BigConstant = 1000000

h = {}
J = {}

for i in range(ClientsNum):
    for j in range(ClientsNum):
        for k in range(CarsNum):
            h[f"x{i}{j}{k}"] = 0

for i in range(ClientsNum):
    for k in range(CarsNum):
        h[f"u{i}{k}"] = 0

for i in range(ClientsNum):
    for k in range(CarsNum):
        h[f"n{i}{k}"] = 0

keys = list(h.keys())

for i in range(len(keys)):
    for j in range(len(keys)):
        if i != j:
            J[(keys[i], keys[j])] = 0

print(h, J)

{'x000': 0, 'x001': 0, 'x010': 0, 'x011': 0, 'x020': 0, 'x021': 0, 'x030': 0, 'x031': 0, 'x040': 0, 'x041': 0, 'x050': 0, 'x051': 0, 'x100': 0, 'x101': 0, 'x110': 0, 'x111': 0, 'x120': 0, 'x121': 0, 'x130': 0, 'x131': 0, 'x140': 0, 'x141': 0, 'x150': 0, 'x151': 0, 'x200': 0, 'x201': 0, 'x210': 0, 'x211': 0, 'x220': 0, 'x221': 0, 'x230': 0, 'x231': 0, 'x240': 0, 'x241': 0, 'x250': 0, 'x251': 0, 'x300': 0, 'x301': 0, 'x310': 0, 'x311': 0, 'x320': 0, 'x321': 0, 'x330': 0, 'x331': 0, 'x340': 0, 'x341': 0, 'x350': 0, 'x351': 0, 'x400': 0, 'x401': 0, 'x410': 0, 'x411': 0, 'x420': 0, 'x421': 0, 'x430': 0, 'x431': 0, 'x440': 0, 'x441': 0, 'x450': 0, 'x451': 0, 'x500': 0, 'x501': 0, 'x510': 0, 'x511': 0, 'x520': 0, 'x521': 0, 'x530': 0, 'x531': 0, 'x540': 0, 'x541': 0, 'x550': 0, 'x551': 0, 'u00': 0, 'u01': 0, 'u10': 0, 'u11': 0, 'u20': 0, 'u21': 0, 'u30': 0, 'u31': 0, 'u40': 0, 'u41': 0, 'u50': 0, 'u51': 0, 'n00': 0, 'n01': 0, 'n10': 0, 'n11': 0, 'n20': 0, 'n21': 0, 'n30': 0, 'n31': 0, 'n40': 

Заполним матрицу QUBO исходя из ограничений и минимизируемого функционала

Напишем функцию для раскрытия выражения вида $(a_1 + a_2 + ... + a_n)^2$, потому что почти все ограничения к этому сводятся

In [485]:
def square(vals): # data example: [(1, "x1"), (-2, "x3"), (3, "x4")]

    for val in vals:
        if val[1] != "":
            h[val[1]] += val[0] * val[0]
        
    for v1 in vals:
        for v2 in vals:
            if v1[1] == v2[1]:
                continue
            if v1[1] == "" and v2[1] == "":
                continue

            if v1[1] != "" and v2[1] != "":
                if (v1[1], v2[1]) not in J.keys():
                    J[(v1[1], v2[1])] = 0
                J[(v1[1], v2[1])] += v1[0] * v2[0]
            else:
                h[v1[1]+v2[1]] += v1[0] * v2[0]

_Целевая функция:_ Функия описывающая издержки для передвижения всех машин. Данную функцию мы хотим минимизировать.

In [486]:
for k in range(CarsNum):
    for i in range(ClientsNum):
        for j in range(ClientsNum):
            print(f"x{i}{j}{k} += {Distances[i][j]}")
            h[f"x{i}{j}{k}"] += Distances[i][j]

for k in range(CarsNum):
    for i in range (ClientsNum):
        for d in range(DepotsNum):
            print(f"u{i}{k} += {Distances[i][ClientsNum + d]} * {CarDepot[k][d]}")
            h[f"u{i}{k}"] += Distances[i][ClientsNum + d] * CarDepot[k][d]

for k in range(CarsNum):
    for i in range (ClientsNum):
        for d in range(DepotsNum):
            print(f"n{i}{k} += {Distances[i][ClientsNum + d]} * {CarDepot[k][d]}")
            h[f"n{i}{k}"] += Distances[i][ClientsNum + d] * CarDepot[k][d]

x000 += 0
x010 += 3959
x020 += 3775
x030 += 2573
x040 += 4108
x050 += 4018
x100 += 3959
x110 += 0
x120 += 3447
x130 += 3751
x140 += 652
x150 += 4324
x200 += 3775
x210 += 3447
x220 += 0
x230 += 2151
x240 += 3702
x250 += 2070
x300 += 2573
x310 += 3751
x320 += 2151
x330 += 0
x340 += 1783
x350 += 1123
x400 += 4108
x410 += 652
x420 += 3702
x430 += 1783
x440 += 0
x450 += 4995
x500 += 4018
x510 += 4324
x520 += 2070
x530 += 1123
x540 += 4995
x550 += 0
x001 += 0
x011 += 3959
x021 += 3775
x031 += 2573
x041 += 4108
x051 += 4018
x101 += 3959
x111 += 0
x121 += 3447
x131 += 3751
x141 += 652
x151 += 4324
x201 += 3775
x211 += 3447
x221 += 0
x231 += 2151
x241 += 3702
x251 += 2070
x301 += 2573
x311 += 3751
x321 += 2151
x331 += 0
x341 += 1783
x351 += 1123
x401 += 4108
x411 += 652
x421 += 3702
x431 += 1783
x441 += 0
x451 += 4995
x501 += 4018
x511 += 4324
x521 += 2070
x531 += 1123
x541 += 4995
x551 += 0
u00 += 665 * 1
u10 += 895 * 1
u20 += 327 * 1
u30 += 3988 * 1
u40 += 3924 * 1
u50 += 1047 * 1
u01 += 665 

_Ограничение 1:_ Нельзя поехать к тому же покупателю.

In [487]:
for i in range(ClientsNum):
    for k in range(CarsNum):
        print(f"x{i}{i}{k} += {BigConstant}")
        h[f"x{i}{i}{k}"] += BigConstant

x000 += 1000000
x001 += 1000000
x110 += 1000000
x111 += 1000000
x220 += 1000000
x221 += 1000000
x330 += 1000000
x331 += 1000000
x440 += 1000000
x441 += 1000000
x550 += 1000000
x551 += 1000000


_Ограничение 2:_ К каждому покупателю должна приехать ровно одна машина.

In [488]:
for i in range(ClientsNum):

    vals = []

    vals.append((BigConstant, ""))

    for k in range(CarsNum):
        for j in range(ClientsNum):
            vals.append((-BigConstant, f"x{j}{i}{k}"))
        vals.append((-BigConstant, f"u{i}{k}"))

    square(vals)

_Ограничение 3:_ От каждого покупателя должна уехать ровно одна машина.

In [489]:
for i in range(ClientsNum):

    vals = []

    vals.append((BigConstant, ""))

    for k in range(CarsNum):
        for j in range(ClientsNum):
            vals.append((-BigConstant, f"x{i}{j}{k}"))
        vals.append((-BigConstant, f"n{i}{k}"))

    square(vals)

_Ограничение 4:_ Каждая машина должна отъехать ровно с одного склада.

In [490]:
for k in range(CarsNum):

    vals = []

    vals.append((BigConstant, ""))

    for i in range(ClientsNum):
        vals.append((-BigConstant, f"u{i}{k}"))

    square(vals)

_Ограничение 5:_ Каждая машина должна приехать ровно на один склад.

In [491]:
for k in range(CarsNum):

    vals = []

    vals.append((BigConstant, ""))

    for i in range(ClientsNum):
        vals.append((-BigConstant, f"n{i}{k}"))

    square(vals)

_Ограничение 6:_ "Целостность маршрута". Если машина приехала к покупателю, то она должна уехать от него.

In [492]:
for i in range(ClientsNum):
    for k in range(CarsNum):
        
        vals = []

        vals.append((BigConstant, f"u{i}{k}"))
        vals.append((-BigConstant, f"n{i}{k}"))

        for j in range(ClientsNum):
            vals.append((BigConstant, f"x{j}{i}{k}"))
            vals.append((-BigConstant, f"x{i}{j}{k}"))

        square(vals)

_Ограничение 7:_ У машин не должно быть возможности иметь на своем маршруте цикл из городов, который они не посещают.

In [493]:
def get_subsets(n):
    res = []
    for i in range(1 << n):
        cur = []
        for j in range(n):
            if (i & (1 << j)):
                cur.append(j)
        res.append(cur)
    return res

id = 0

for S in get_subsets(ClientsNum):
    if len(S) >= 2:

        vals = []

        vals.append((BigConstant * (-len(S) + 1), ""))

        for k in range(CarsNum):
            for i in S:
                for j in S:
                    if i != j:
                        vals.append((BigConstant, f"x{i}{j}{k}"))

        for l in range(math.ceil(math.log2(len(S)))):
            
            id += 1
            h[f"l{id}"] = 0
            vals.append((2**l * BigConstant, f"l{id}"))

        square(vals)

Теперь надо привести матрицу к верхнетреугольному виду

In [494]:
print(len(h), len(J))

for k1 in h.keys():
    for k2 in h.keys():
        if k1 != k2 and (k1, k2) in J.keys() and (k2, k1) in J.keys():
            J[(k1, k2)] += J[(k2, k1)]
            del J[(k2, k1)]

print(len(h), len(J))
print(h)
print(J)

202 13552
202 6776
{'x000': 1000000, 'x001': 1000000, 'x010': -79999999996041, 'x011': -79999999996041, 'x020': -79999999996225, 'x021': -79999999996225, 'x030': -79999999997427, 'x031': -79999999997427, 'x040': -79999999995892, 'x041': -79999999995892, 'x050': -79999999995982, 'x051': -79999999995982, 'x100': -79999999996041, 'x101': -79999999996041, 'x110': 1000000, 'x111': 1000000, 'x120': -79999999996553, 'x121': -79999999996553, 'x130': -79999999996249, 'x131': -79999999996249, 'x140': -79999999999348, 'x141': -79999999999348, 'x150': -79999999995676, 'x151': -79999999995676, 'x200': -79999999996225, 'x201': -79999999996225, 'x210': -79999999996553, 'x211': -79999999996553, 'x220': 1000000, 'x221': 1000000, 'x230': -79999999997849, 'x231': -79999999997849, 'x240': -79999999996298, 'x241': -79999999996298, 'x250': -79999999997930, 'x251': -79999999997930, 'x300': -79999999997427, 'x301': -79999999997427, 'x310': -79999999996249, 'x311': -79999999996249, 'x320': -79999999997849, 'x3

Запустим квантовый отжиг

In [495]:
bqm = BinaryQuadraticModel(h, J, 'BINARY')

min_energy = 0
answer = {}

# Exact solver


# solution = ExactSolver().sample(bqm)
# print("ANSWER:", solution.first.energy)
# print(solution.first.sample)

# min_energy = solution.first.energy
# answer = solution.first.sample


# End Exact solver

for _ in range(10000):
    sample = SimulatedAnnealingSampler().sample(bqm, num_reads=1)
    if sample.first.energy < min_energy:
        min_energy = sample.first.energy
        answer = sample.first.sample

print("Min energy:", min_energy)
print("Optimizing parameters amount:", len(h.keys()))

total_distance = 0

for k in range(CarsNum):

    for i in range(ClientsNum):
        if answer[f"u{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: Depot -> {i}, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")

    for i in range(ClientsNum):
        for j in range(ClientsNum):
            if answer[f"x{i}{j}{k}"] == 1:
                total_distance += Distances[i][j]
                print(f"Car {k}: {i} -> {j}, distance: {Distances[i][j]}")

    for i in range(ClientsNum):
        if answer[f"n{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: {i} -> Depot, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")


print("Answer: ", total_distance)

Min energy: -366999999990298.0
Optimizing parameters amount: 202
Car 0: Depot -> 5, distance: 1047
Car 0: 1 -> 2, distance: 3447
Car 0: 3 -> 4, distance: 1783
Car 0: 4 -> 1, distance: 652
Car 0: 5 -> 3, distance: 1123
Car 0: 2 -> Depot, distance: 327
Car 1: Depot -> 0, distance: 665
Car 1: 0 -> Depot, distance: 665
Answer:  9709


Давайте вычислим ответ полным перебором

In [496]:
answer_complete_search = BigConstant

for perm in itertools.permutations(list(range(ClientsNum))):
    dist = Distances[perm[0]][ClientsNum] + Distances[perm[-1]][ClientsNum]
    for i in range(len(perm) - 1):
        dist += Distances[perm[i]][perm[i + 1]]

    answer_complete_search = min(answer_complete_search, dist)

print("Answer Complete Search:", answer_complete_search)

Answer Complete Search: 10579


Решим задачу используя QDeepSDK

In [499]:
from qdeepsdk import QDeepHybridSolver
import numpy as np

solver = QDeepHybridSolver()
solver.m_budget = 1000
solver.num_reads = 5000
solver.token = "hf6si03meu"

keys_array = list(h.keys())
keys_map = {}
for i in range(len(keys_array)):
    keys_map[keys_array[i]] = i

matrix = np.zeros((len(keys_array), len(keys_array)), dtype=int)

for key in h:
    matrix[keys_map[key]][keys_map[key]] = h[key]

for key in J:
    matrix[keys_map[key[0]]][keys_map[key[1]]] = J[key]

resp = solver.solve(matrix)
res = resp['QdeepHybridSolver']['configuration']

print("Energy: ", resp['QdeepHybridSolver']['energy'])

print("Parameters amount:", len(res))

total_distance = 0

for k in range(CarsNum):

    for i in range(ClientsNum):
        if res[keys_map[f"u{i}{k}"]] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: Depot -> {i}, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")

    for i in range(ClientsNum):
        for j in range(ClientsNum):
            if res[keys_map[f"x{i}{j}{k}"]] == 1:
                total_distance += Distances[i][j]
                print(f"Car {k}: {i} -> {j}, distance: {Distances[i][j]}")

    for i in range(ClientsNum):
        if res[keys_map[f"n{i}{k}"]] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: {i} -> Depot, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")


print("Answer QDeep: ", total_distance)

Energy:  -330000020537344
Parameters amount: 202
Car 0: Depot -> 2, distance: 327
Car 0: 2 -> 5, distance: 2070
Car 0: 5 -> 3, distance: 1123
Car 0: 3 -> Depot, distance: 3988
Car 1: Depot -> 0, distance: 665
Car 1: 0 -> Depot, distance: 665
Answer QDeep:  8838
